In [2]:
# JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [3]:
bnf_1hr = pd.read_csv("../data/gold_4hr_tv (2).csv")
# bnf_1hr = pd.read_csv("../data/gold_1hr_tv.csv")
# bnf_1hr = pd.read_csv("../data/midcp_select_1hr_tv (4).csv")
# bnf_1hr = pd.read_csv("../data/fnf_1hr_tv.csv")
bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
bnf_1hr = bnf_1hr.drop(columns=["time"])
bnf_1hr = bnf_1hr[(bnf_1hr["datetime"].dt.year >= 2017)]
bnf_1hr.tail()

,open,high,low,close,datetime
9361,72650,72660,72130,72343,2024-07-08 21:00:00+05:30
9362,72577,72598,72310,72316,2024-07-09 09:00:00+05:30
9363,72327,72506,72311,72385,2024-07-09 13:00:00+05:30
9364,72377,72634,72111,72200,2024-07-09 17:00:00+05:30
9365,72217,72450,72191,72432,2024-07-09 21:00:00+05:30


In [4]:
# spot_data = pd.read_csv('../data/bnf.csv')
# spot_data['datetime'] = pd.to_datetime(spot_data['datetime'])
# spot_data = pl.DataFrame(spot_data)
# spot_data = spot_data.with_columns([pl.col('datetime').alias('index')])

# def resample(
#     data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
# ) -> pl.DataFrame:
#     return (
#         data.set_sorted("datetime")
#         .group_by_dynamic(
#             index_column="datetime",
#             every=timeframe,
#             period=timeframe,
#             label='left',
#             offset=offset,
#         )
#         .agg(
#             [
#                 pl.col("open").first().alias("open"),
#                 pl.col("high").max().alias("high"),
#                 pl.col("low").min().alias("low"),
#                 pl.col("close").last().alias("close"),
#                 pl.col("volume").sum().alias("volume"),
#             ]
#         )
#     )

# bnf_resampled = resample(spot_data, '5m', pd.Timedelta(minutes=0))

# bnf_df = bnf_resampled.to_pandas()
# bnf_1hr = bnf_df

In [5]:
bnf_1hr.tail()

,open,high,low,close,datetime
9361,72650,72660,72130,72343,2024-07-08 21:00:00+05:30
9362,72577,72598,72310,72316,2024-07-09 09:00:00+05:30
9363,72327,72506,72311,72385,2024-07-09 13:00:00+05:30
9364,72377,72634,72111,72200,2024-07-09 17:00:00+05:30
9365,72217,72450,72191,72432,2024-07-09 21:00:00+05:30


In [6]:
PORTFOLIO = 10_00_000
INDEX_LEVERAGE = 10
SLIPPAGE_FACTOR = 0.0003

In [7]:
def generate_pivots2(df):

    df["Bullish Pivot"] = 0
    df["Bearish Pivot"] = 0
    # df["MA"] = df["close"].rolling(window=18).mean()

    # Generate signals using boolean masking
    bull_pivot_mask = (df["low"] >= df["low"].shift(1)) & (
        df["low"].shift(2) >= df["low"].shift(1)
    )
    bear_pivot_mask = (df["high"] <= df["high"].shift(1)) & (
        df["high"].shift(2) <= df["high"].shift(1)
    )

    df.loc[bull_pivot_mask, "Bullish Pivot"] = 1
    df.loc[bear_pivot_mask, "Bearish Pivot"] = 1

    return df


bnf_1hr = generate_pivots2(bnf_1hr)
bnf_1hr.head()

,open,high,low,close,datetime,Bullish Pivot,Bearish Pivot
1844,27401,27591,27401,27500,2017-01-02 09:00:00+05:30,0,0
1845,27500,27546,27496,27540,2017-01-02 13:00:00+05:30,0,0
1846,27540,27575,27533,27548,2017-01-02 17:00:00+05:30,0,0
1847,27550,27600,27535,27560,2017-01-02 21:00:00+05:30,0,0
1848,27568,27628,27520,27526,2017-01-03 09:00:00+05:30,0,0


In [8]:
def add_ma(df, ma_period=60):
    df['MA'] = df['close'].rolling(window=ma_period).mean()
    return df

bnf_1hr = add_ma(bnf_1hr)

In [9]:
def calculate_rsi(df, column="close", period=5):

    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


bnf_1hr["RSI"] = calculate_rsi(bnf_1hr)

In [10]:
percentiles = [0.1, 0.2, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

percentile_values = bnf_1hr['RSI'].quantile(percentiles)

print("Percentile Distribution of 'High Entry Ratio':")
print(percentile_values)

Percentile Distribution of 'High Entry Ratio':
0.1000    12.6437
0.2000    24.4224
0.2500    29.8246
0.5000    52.7311
0.7500    75.0973
0.9000    91.2752
0.9500    98.0609
0.9900   100.0000
Name: RSI, dtype: float64


In [152]:
def execute_trades_long(df):
    tradebook = []

    in_trade = False
    long_trade_active = False
    short_trade_active = False

    trailing_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    long_sl_pct = 1.5
    long_target_pct = 100

    rsi_upper_range = 89
    rsi_lower_range = 20

    long_points = 0
    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i - 1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        print(current_candle_time, df.iloc[i]['RSI'])

        if not in_trade:
            if df.iloc[i - 1]["Bullish Pivot"] == 1:
                # Long Side
                signal_time = previous_candle_time

                if current_candle_high >= previous_candle_high and (
                    df.iloc[i - 1]["RSI"] <= rsi_lower_range
                ):
                    if (current_candle_open > previous_candle_high) and (current_candle_time.date() > previous_candle_time.date()):
                        # Check Entry Skip on Next Day Gap Up Open
                        # print("Long Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        # print("Long Entry Triggered", current_candle_time)
                        in_trade = True
                        long_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_high
                        # initial_sl = min(
                        #     df.iloc[i - 2]["low"], entry_price * (1 - long_sl_pct / 100)
                        # )
                        # initial_sl = df.iloc[i-2]['low']
                        initial_sl = entry_price * (1 - long_sl_pct / 100)
                        long_target = entry_price * (1 + long_target_pct / 100)
                        # long_target = entry_price + (1.75 * (entry_price - initial_sl))
                        # qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                        qty = 0.1 * PORTFOLIO / abs(entry_price - initial_sl)

        if in_trade:
            if long_trade_active:
                
                if current_candle_low > df.iloc[i]['MA']:
                    trailing_active = True

                if trailing_active:

                    trailing_ma = df.iloc[i]['MA']
                    
                    if current_candle_open < initial_sl:
                        # Gap Outside Initial SL
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        long_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)

                    # elif current_candle_close <= trailing_ma:
                    #     #TSL Hit
                    #     in_trade = False
                    #     long_trade_active = False
                    #     exit_price = current_candle_close
                    #     exit_time = current_candle_time
                    #     long_points = exit_price - entry_price
                    #     remark = "Trailing SL Hit"
                    #     # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= long_target:
                        in_trade = False
                        long_trade_active = False
                        exit_price = long_target
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] >= rsi_upper_range) and (df.iloc[i]["RSI"] <= rsi_upper_range):
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "RSI Overbought"

                else:
                    
                    if current_candle_open < initial_sl:
                        # Gap Outside Initial SL
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        long_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= long_target:
                        in_trade = False
                        long_trade_active = False
                        exit_price = long_target
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] >= rsi_upper_range) and (df.iloc[i]["RSI"] <= rsi_upper_range):
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "RSI Overbought"
                        
                if long_points:
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    long_points = 0
                    long_trade_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df

In [153]:
tb_long = execute_trades_long(bnf_1hr)

2017-01-02 09:00:00+05:30 nan
2017-01-02 13:00:00+05:30 100.0
2017-01-02 17:00:00+05:30 100.0
2017-01-02 21:00:00+05:30 100.0
2017-01-03 09:00:00+05:30 63.829787234042556
2017-01-03 13:00:00+05:30 37.97468354430379
2017-01-03 17:00:00+05:30 53.55450236966824
2017-01-03 21:00:00+05:30 75.12690355329948
2017-01-04 09:00:00+05:30 68.10551558752996
2017-01-04 13:00:00+05:30 77.13625866050808
2017-01-04 17:00:00+05:30 79.33491686460808
2017-01-04 21:00:00+05:30 68.4659090909091
2017-01-05 09:00:00+05:30 68.82022471910112
2017-01-05 13:00:00+05:30 65.15957446808511
2017-01-05 17:00:00+05:30 68.95734597156398
2017-01-05 21:00:00+05:30 80.0
2017-01-06 09:00:00+05:30 67.09129511677281
2017-01-06 13:00:00+05:30 50.9493670886076
2017-01-06 17:00:00+05:30 62.264150943396224
2017-01-06 21:00:00+05:30 41.860465116279066
2017-01-09 09:00:00+05:30 25.683060109289627
2017-01-09 13:00:00+05:30 78.3132530120482
2017-01-09 17:00:00+05:30 59.60264900662252
2017-01-09 21:00:00+05:30 79.46127946127946
2017-0

In [154]:
tb_long['ROI% w CS'].sum()

381.70596519426545

In [155]:
tb_targets = tb_long[tb_long['Remarks'] == 'Target Hit']
tb_rsi_overbought = tb_long[tb_long['Remarks'] == 'RSI Overbought']
tb_sl = tb_long[tb_long['Remarks'] == 'Initial SL Hit']
len(tb_targets) , len(tb_rsi_overbought) , len(tb_sl)

(0, 89, 16)

In [156]:
stats_df5 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)

new_tb = tb_long

# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = new_tb[(new_tb["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df5.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df5["Total ROI"].sum()
overall_total_trades = stats_df5["Total Trades"].sum()
overall_win_rate = (new_tb["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = new_tb[new_tb["ROI% w CS"] > 0]["ROI% w CS"].mean()
overall_avg_loss = new_tb[new_tb["ROI% w CS"] < 0]["ROI% w CS"].mean()
overall_max_drawdown = (
    new_tb["ROI% w CS"].cumsum() - new_tb["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df5.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df5

/tmp/ipykernel_8917/1507484520.py:42: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,29.7107,14,71.4286,5.4423,-6.1781,-10.3970,2.8576
2018,18.3219,12,75.0000,4.5306,-7.4844,-12.8792,1.4226
2019,73.8335,14,71.4286,9.2371,-4.6345,-14.0166,5.2676
2020,85.5336,19,78.9474,7.8127,-7.9141,-13.5088,6.3317
2021,58.5344,15,80.0000,7.4771,-10.3970,-10.3970,5.6299
2022,35.5412,12,66.6667,8.3835,-7.8816,-10.7324,3.3116
2023,56.9120,16,68.7500,8.6963,-7.7494,-17.9225,3.1754
2024,23.3186,4,75.0000,7.9097,-0.4105,0.0000,inf
Overall,381.7060,106.0000,73.5849,7.4480,-7.1155,-20.7940,18.3565


# GOLD Buy The Dip System -> RSI(4) + Pivots (95, 30) 1.5 : 5

In [157]:
tb_long

,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year
0,LONG,2017-01-27 13:00:00+05:30,2017-01-27 17:00:00+05:30,28168,27745.4800,2017-02-01 13:00:00+05:30,28817.0000,RSI Overbought,649.0000,631.9045,236.6752,10,153602.1963,149556.1157,15.3602,14.9556,2017
1,LONG,2017-05-02 13:00:00+05:30,2017-05-02 17:00:00+05:30,28630,28200.5500,2017-05-04 09:00:00+05:30,28200.5500,Initial SL Hit,-429.4500,-446.4992,232.8560,10,-100000.0000,-103970.0000,-10.0000,-10.3970,2017
2,LONG,2017-05-04 21:00:00+05:30,2017-05-05 09:00:00+05:30,28155,27732.6750,2017-05-18 17:00:00+05:30,28771.0000,RSI Overbought,616.0000,598.9222,236.7845,10,145859.2316,141815.4739,14.5859,14.1815,2017
3,LONG,2017-05-24 13:00:00+05:30,2017-05-24 17:00:00+05:30,28720,28289.2000,2017-05-25 17:00:00+05:30,28649.0000,RSI Overbought,-71.0000,-88.2107,232.1263,10,-16480.9656,-20476.0214,-1.6481,-2.0476,2017
4,LONG,2017-06-02 13:00:00+05:30,2017-06-02 17:00:00+05:30,28884,28450.7400,2017-06-05 21:00:00+05:30,29205.0000,RSI Overbought,321.0000,303.5733,230.8083,10,74089.4613,70067.2345,7.4089,7.0067,2017
5,LONG,2017-06-15 21:00:00+05:30,2017-06-16 09:00:00+05:30,28785,28353.2250,2017-06-26 17:00:00+05:30,28353.2250,Initial SL Hit,-431.7750,-448.9165,231.6021,10,-100000.0000,-103970.0000,-10.0000,-10.3970,2017
6,LONG,2017-07-10 13:00:00+05:30,2017-07-10 17:00:00+05:30,27734,27317.9900,2017-07-12 21:00:00+05:30,27842.0000,RSI Overbought,108.0000,91.3272,240.3788,10,25960.9144,21953.1261,2.5961,2.1953,2017
7,LONG,2017-07-26 13:00:00+05:30,2017-07-26 17:00:00+05:30,28350,27924.7500,2017-07-27 17:00:00+05:30,28450.0000,RSI Overbought,100.0000,82.9600,235.1558,10,23515.5791,19508.5244,2.3516,1.9509,2017
8,LONG,2017-09-07 09:00:00+05:30,2017-09-07 13:00:00+05:30,30120,29668.2000,2017-09-08 13:00:00+05:30,30364.0000,RSI Overbought,244.0000,225.8548,221.3369,10,54006.1974,49989.9956,5.4006,4.9990,2017
9,LONG,2017-09-12 13:00:00+05:30,2017-09-12 17:00:00+05:30,29908,29459.3800,2017-09-13 17:00:00+05:30,29842.0000,RSI Overbought,-66.0000,-83.9250,222.9058,10,-14711.7828,-18707.3693,-1.4712,-1.8707,2017


In [158]:
tb_long['Prof Loss Remark'] = tb_long['ROI%'].apply(lambda x: 'Profitable' if x > 0 else 'Lossy')

In [159]:
df = bnf_1hr.copy()
# tb_long['Max High'] = 0

for i in range(0, len(tb_long)):
    entry_time = tb_long.iloc[i]['Entry Time']
    exit_time = tb_long.iloc[i]['Exit Time']
    subset_df = df[(df['datetime'] >= entry_time) & (df['datetime'] <= exit_time)]
    max_high = subset_df['high'].max()
    # print(max_high)
    tb_long.at[i, 'Max High'] = int(max_high)
    tb_long.at[i, 'Max Unrealised Profit'] = (tb_long.iloc[i]['Max High'] - tb_long.iloc[i]['Entry Price']) * tb_long.iloc[i]['Qty']
    tb_long.at[i, 'Value At Risk'] = (tb_long.iloc[i]['Entry Price'] - tb_long.iloc[i]['Initial SL']) * tb_long.iloc[i]['Qty']
    tb_long.at[i, 'Max nR'] = tb_long.iloc[i]['Max Unrealised Profit'] / tb_long.iloc[i]['Value At Risk']
    tb_long.at[i, 'Booked nR'] = tb_long.iloc[i]['ROI%']/10
    tb_long.at[i, 'Booked nR w CS'] = tb_long.iloc[i]['ROI% w CS']/10

# tb_long

In [160]:
tb_long

,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year,Prof Loss Remark,Max High,Max Unrealised Profit,Value At Risk,Max nR,Booked nR,Booked nR w CS
0,LONG,2017-01-27 13:00:00+05:30,2017-01-27 17:00:00+05:30,28168,27745.4800,2017-02-01 13:00:00+05:30,28817.0000,RSI Overbought,649.0000,631.9045,236.6752,10,153602.1963,149556.1157,15.3602,14.9556,2017,Profitable,28966.0000,188866.7992,100000.0000,1.8887,1.5360,1.4956
1,LONG,2017-05-02 13:00:00+05:30,2017-05-02 17:00:00+05:30,28630,28200.5500,2017-05-04 09:00:00+05:30,28200.5500,Initial SL Hit,-429.4500,-446.4992,232.8560,10,-100000.0000,-103970.0000,-10.0000,-10.3970,2017,Lossy,28634.0000,931.4239,100000.0000,0.0093,-1.0000,-1.0397
2,LONG,2017-05-04 21:00:00+05:30,2017-05-05 09:00:00+05:30,28155,27732.6750,2017-05-18 17:00:00+05:30,28771.0000,RSI Overbought,616.0000,598.9222,236.7845,10,145859.2316,141815.4739,14.5859,14.1815,2017,Profitable,29004.0000,201030.0124,100000.0000,2.0103,1.4586,1.4182
3,LONG,2017-05-24 13:00:00+05:30,2017-05-24 17:00:00+05:30,28720,28289.2000,2017-05-25 17:00:00+05:30,28649.0000,RSI Overbought,-71.0000,-88.2107,232.1263,10,-16480.9656,-20476.0214,-1.6481,-2.0476,2017,Lossy,28765.0000,10445.6825,100000.0000,0.1045,-0.1648,-0.2048
4,LONG,2017-06-02 13:00:00+05:30,2017-06-02 17:00:00+05:30,28884,28450.7400,2017-06-05 21:00:00+05:30,29205.0000,RSI Overbought,321.0000,303.5733,230.8083,10,74089.4613,70067.2345,7.4089,7.0067,2017,Profitable,29299.0000,95785.4406,100000.0000,0.9579,0.7409,0.7007
5,LONG,2017-06-15 21:00:00+05:30,2017-06-16 09:00:00+05:30,28785,28353.2250,2017-06-26 17:00:00+05:30,28353.2250,Initial SL Hit,-431.7750,-448.9165,231.6021,10,-100000.0000,-103970.0000,-10.0000,-10.3970,2017,Lossy,28820.0000,8106.0738,100000.0000,0.0811,-1.0000,-1.0397
6,LONG,2017-07-10 13:00:00+05:30,2017-07-10 17:00:00+05:30,27734,27317.9900,2017-07-12 21:00:00+05:30,27842.0000,RSI Overbought,108.0000,91.3272,240.3788,10,25960.9144,21953.1261,2.5961,2.1953,2017,Profitable,27977.0000,58412.0574,100000.0000,0.5841,0.2596,0.2195
7,LONG,2017-07-26 13:00:00+05:30,2017-07-26 17:00:00+05:30,28350,27924.7500,2017-07-27 17:00:00+05:30,28450.0000,RSI Overbought,100.0000,82.9600,235.1558,10,23515.5791,19508.5244,2.3516,1.9509,2017,Profitable,28607.0000,60435.0382,100000.0000,0.6044,0.2352,0.1951
8,LONG,2017-09-07 09:00:00+05:30,2017-09-07 13:00:00+05:30,30120,29668.2000,2017-09-08 13:00:00+05:30,30364.0000,RSI Overbought,244.0000,225.8548,221.3369,10,54006.1974,49989.9956,5.4006,4.9990,2017,Profitable,30474.0000,78353.2537,100000.0000,0.7835,0.5401,0.4999
9,LONG,2017-09-12 13:00:00+05:30,2017-09-12 17:00:00+05:30,29908,29459.3800,2017-09-13 17:00:00+05:30,29842.0000,RSI Overbought,-66.0000,-83.9250,222.9058,10,-14711.7828,-18707.3693,-1.4712,-1.8707,2017,Lossy,30058.0000,33435.8700,100000.0000,0.3344,-0.1471,-0.1871


In [161]:
percentiles = [0.1, 0.2, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

percentile_values = tb_long['Max nR'].quantile(percentiles)

print(percentile_values)

0.1000   0.1409
0.2000   0.3670
0.2500   0.4269
0.5000   0.7681
0.7500   1.2481
0.9000   1.7899
0.9500   2.0090
0.9900   3.2460
Name: Max nR, dtype: float64


In [44]:
# tb_long.to_csv('Parle-G Long 5_89_20 1pt5 sl.csv')

In [162]:
def calculate_rsi_short(df, column="close", period=5):

    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


bnf_1hr["RSI"] = calculate_rsi_short(bnf_1hr)

In [163]:
percentiles = [0.1, 0.2, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

percentile_values = bnf_1hr['RSI'].quantile(percentiles)

print("Percentile Distribution of 'High Entry Ratio':")
print(percentile_values)

Percentile Distribution of 'High Entry Ratio':
0.1000    12.6437
0.2000    24.4224
0.2500    29.8246
0.5000    52.7311
0.7500    75.0973
0.9000    91.2752
0.9500    98.0609
0.9900   100.0000
Name: RSI, dtype: float64


In [189]:
def execute_trades_short(df):
    tradebook = []

    in_trade = False
    short_trade_active = False

    trailing_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    short_sl_pct = 1.50
    short_target_pct = 100

    rsi_upper_range = 75
    rsi_lower_range = 11

    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i - 1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        print(current_candle_time, df.iloc[i]['RSI'])

        if not in_trade:
            if df.iloc[i - 1]["Bearish Pivot"] == 1:
                # Short Side
                signal_time = previous_candle_time

                if current_candle_low <= previous_candle_low and (
                    df.iloc[i - 1]["RSI"] >= rsi_upper_range
                ):
                    if (current_candle_open < previous_candle_low) and (current_candle_time.date() > previous_candle_time.date()):
                        # Check Entry Skip on Next Day Gap Up Open
                        # print("Short Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        # print("Short Entry Triggered", current_candle_time)
                        in_trade = True
                        short_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_low
                        # initial_sl = min(
                        #     df.iloc[i - 2]["low"], entry_price * (1 - short_sl_pct / 100)
                        # )
                        initial_sl = entry_price * (1 + short_sl_pct / 100)
                        short_target = entry_price * (1 - short_target_pct / 100)
                        qty = PORTFOLIO * INDEX_LEVERAGE / entry_price

        if in_trade:
            if short_trade_active:
                
                if current_candle_high < df.iloc[i]['MA']:
                    trailing_active = True

                if trailing_active:

                    trailing_ma = df.iloc[i]['MA']
                    
                    if current_candle_open > initial_sl:
                        # Gap Outside Initial SL
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        short_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)

                    elif current_candle_close <= trailing_ma:
                        #TSL Hit
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        short_points = exit_price - entry_price
                        remark = "Trailing SL Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= short_target:
                        in_trade = False
                        short_trade_active = False
                        exit_price = short_target
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] <= rsi_lower_range) and (df.iloc[i]["RSI"] >= rsi_lower_range):
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "RSI Oversold"

                else:
                    
                    if current_candle_open > initial_sl:
                        # Gap Outside Initial SL
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        short_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= short_target:
                        in_trade = False
                        short_trade_active = False
                        exit_price = short_target
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] <= rsi_lower_range) and (df.iloc[i]["RSI"] >= rsi_lower_range):
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "RSI Oversold"
                        
                if short_points:
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (short_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0
                    short_trade_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df

In [190]:
tb_short = execute_trades_short(bnf_1hr)

2017-01-02 09:00:00+05:30 nan
2017-01-02 13:00:00+05:30 100.0
2017-01-02 17:00:00+05:30 100.0
2017-01-02 21:00:00+05:30 100.0
2017-01-03 09:00:00+05:30 63.829787234042556
2017-01-03 13:00:00+05:30 37.97468354430379
2017-01-03 17:00:00+05:30 53.55450236966824
2017-01-03 21:00:00+05:30 75.12690355329948
2017-01-04 09:00:00+05:30 68.10551558752996
2017-01-04 13:00:00+05:30 77.13625866050808
2017-01-04 17:00:00+05:30 79.33491686460808
2017-01-04 21:00:00+05:30 68.4659090909091
2017-01-05 09:00:00+05:30 68.82022471910112
2017-01-05 13:00:00+05:30 65.15957446808511
2017-01-05 17:00:00+05:30 68.95734597156398
2017-01-05 21:00:00+05:30 80.0
2017-01-06 09:00:00+05:30 67.09129511677281
2017-01-06 13:00:00+05:30 50.9493670886076
2017-01-06 17:00:00+05:30 62.264150943396224
2017-01-06 21:00:00+05:30 41.860465116279066
2017-01-09 09:00:00+05:30 25.683060109289627
2017-01-09 13:00:00+05:30 78.3132530120482
2017-01-09 17:00:00+05:30 59.60264900662252
2017-01-09 21:00:00+05:30 79.46127946127946
2017-0

In [191]:
tb_targets_short = tb_short[tb_short['Remarks'] == 'Target Hit']
tb_rsi_oversold_short = tb_short[tb_short['Remarks'] == 'RSI Oversold']
tb_sl_short = tb_short[tb_short['Remarks'] == 'Initial SL Hit']
len(tb_targets_short) , len(tb_rsi_oversold_short) , len(tb_sl_short)

(0, 38, 42)

In [192]:
tb_short['ROI% w CS'].sum()

-935.8620900587609

In [193]:
# tb_short.tail(25)

In [194]:
stats_df5 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)

new_tb = tb_short

# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = new_tb[(new_tb["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df5.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df5["Total ROI"].sum()
overall_total_trades = stats_df5["Total Trades"].sum()
overall_win_rate = (new_tb["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = new_tb[new_tb["ROI% w CS"] > 0]["ROI% w CS"].mean()
overall_avg_loss = new_tb[new_tb["ROI% w CS"] < 0]["ROI% w CS"].mean()
overall_max_drawdown = (
    new_tb["ROI% w CS"].cumsum() - new_tb["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df5.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df5

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,-101.6696,24,25.0000,4.5228,-7.1559,-86.0651,-1.1813
2018,-131.5800,23,13.0435,3.8523,-7.1568,-124.2976,-1.0586
2019,-127.1177,26,34.6154,5.7961,-10.5461,-123.5482,-1.0289
2020,-141.2784,23,30.4348,7.5311,-12.1247,-161.8078,-0.8731
2021,-100.8768,17,23.5294,5.7452,-9.5275,-75.7609,-1.3315
2022,-109.3390,17,17.6471,3.0487,-8.4632,-93.7345,-1.1665
2023,-78.9118,22,31.8182,6.3399,-8.2194,-89.0930,-0.8857
2024,-145.0889,13,7.6923,0.3968,-12.1238,-140.5345,-1.0324
Overall,-935.8621,165.0000,24.2424,5.5120,-9.2507,-920.2576,-1.0170
